<center><h1><b>Seam Carving Algorithm With Python</b></h1></center>

<b>Seam carving: </b> also known as content-aware image resizing, is one of the methods 
for adjusting the image size such that the content of the image will be preserved.
Image resizing is used to handle different resolutions between modules of various 
device screens and applications. It can also be used intentionally to corrupt or 
remove the original content of the image. Therefore, it is of great importance 
to detect external interference with the images.

In [ ]:
import cv2
import numpy as np
import sys
import glob
import os, shutil
import glob
from os import listdir
from os.path import isfile, join
#from matplotlib import pyplot as plt
from skimage.filters import prewitt,roberts,scharr
from skimage.feature import canny
from scipy.ndimage.filters import convolve
from os import path

In [2]:
def seamcarveVertical(I, K):
    # silinecek piksel için bir sütunluk ye ayır
    pix = np.zeros((I.shape[0], 1))
    # diziyi bir kere oluştur boyutunu küçült
    r = np.arange(0, I.shape[0]).reshape(I.shape[0], 1)

    for k in range(1, K + 1):
        griI = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
        # G=cv2.Laplacian(griI,cv2.CV_64F,ksize=3)
        # G= scharr(griI)
        # G= scharr(griI)
        G = canny(griI)
        # G= prewitt(griI)

        # padding
        G[:, 0] = G[:, 1]
        G[:, -1] = G[:, -2]
        G[0, :] = G[1, :]
        G[-1, :] = G[-2, :]
        # plt.figure(1)
        # plt.imshow(G,cmap='gray')
        Pot = G.copy()
        A = []
        for ii in range(1, Pot.shape[0]):
            # padding pp=[Pot(ii-1,1) Pot(ii-1,:)  Pot(ii-1,end)];
            pp = np.concatenate((Pot[ii - 1, 0].reshape(1, ),
                                 Pot[ii - 1, :].reshape(Pot.shape[1], ),
                                 Pot[ii - 1, -1].reshape(1, )), axis=0)
            # her pikselin üstündeki sagındaki ve solundaki minimum piksel
            S = np.minimum(pp[0:-2], pp[1:-1])
            S = np.minimum(pp[2:], S)
            # hesaplanan vektör satıra ekle
            A = np.add(Pot[ii, :], S)
            Pot[ii, :] = A
        # son satırdaki en düşük enerjili sütun
        loc = np.argmin(Pot[-1, :])
        pix = np.zeros((G.shape[0], 1))
        pix[-1] = loc;
        # bottom up
        for ii in range(G.shape[0] - 1, -1, -1):
            if loc > 0 and loc < (G.shape[1] - 1):
                loc = np.argmin(Pot[ii, (loc - 1):(loc + 1 + 1)]) + loc - 1

            elif loc == (G.shape[1] - 1):  # rightmost column
                loc = np.argmin(Pot[ii, (loc - 1):(loc + 1)]) + loc - 1
            else:  # leftmostcolumn
                loc = np.argmin(Pot[ii, loc:(loc + 1 + 1)]) + loc

            pix[ii] = loc

        Sel = np.zeros((G.shape[0], G.shape[1] - 1), dtype=int)
        for i in range(G.shape[0]):
            sel = np.concatenate((np.arange(0, pix[i]),
                                  np.arange(pix[i] + 1, G.shape[1])))
            Sel[i, :] = sel
        I = I[r, Sel, :]

    return I



In [3]:
def seamcarving(I, K):
    B = np.zeros((I.shape[0] - K, I.shape[1], I.shape[2]), dtype=np.uint8)
    # print('1-O=seamcarveVertical(I,K)')
    O = seamcarveVertical(I, K)
    B[:, :, 0] = np.transpose(O[:, :, 0])
    B[:, :, 1] = np.transpose(O[:, :, 1])
    B[:, :, 2] = np.transpose(O[:, :, 2])
    # print('2-B=seamcarveVertical(B,K)')
    O = seamcarveVertical(B, K)
    B = np.zeros((I.shape[0] - K, I.shape[1] - K, I.shape[2]), dtype=np.uint8)  # 'uint8'
    # print('B.shape=',B.shape)
    B[:, :, 0] = np.transpose(O[:, :, 0])
    B[:, :, 1] = np.transpose(O[:, :, 1])
    B[:, :, 2] = np.transpose(O[:, :, 2])
    return B


In [4]:
def seamcarvingbatch(I, K):
    b = I.shape
    O = np.zeros((b[0], b[1] - K, b[2] - K, b[3]), dtype=np.uint8)
    for i in range(b[0]):
        O[i, :, :, :] = seamcarving(I[i, :, :, :].reshape(b[1], b[2], b[3]), K)
    return O


In [6]:
if __name__ == "__main__":
    """seam_num = 5 # percentage of retarget image
    src_dir = "./train_img/"
    dst_dir = "./val_imgs/""""

    dosya = [f for f in listdir(src_dir) if isfile(join(src_dir, f))]
    images = np.empty(len(dosya), dtype=object)
    for n in range(0, len(dosya)):
        images[n] = cv2.imread(join(src_dir, dosya[n]))
        img = seamcarveVertical(images[n], seam_num)
        cv2.imwrite(dst_dir + '{}.jpg'.format(n), img)


## For More Information Click to the Link Below

<a href="https://tez.yok.gov.tr/UlusalTezMerkezi/TezGoster?key=8tbPippmWV_b-Irrn9YEAsnI4wTAxUJFKVY8MOWHdJ-wbT3aEbE4e7I4v4i18Lcq">Click Here</a>